In [11]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model


frame_data = pd.read_excel('FPS_DATA.xlsx')

frame_data.head()

,"SPEC(High, Mid, Low)",CPU NAME,CPU BenchMark,CPU Core,CPU Thread,CPU ClockSpeed GHZ,CPU ClockSpeed Turbo GHz,GPU NAME,GPU BenchMark,GPU Memory GB,GPU Clock MHz,GPU Clock Boost MHz,GAME NAME,RESOLUTION,GAME SETTING,GAME MIN FRAME,GAME AVG FRAME,GAME MAX FRAME
0,High,Intel Core i9-12900,38803,16,24,2.4,5.1,GeForce RTX 3090,26303,24.0,1560,1860,1,QHD,ULTRA,71,87,133
1,High,Intel Core i9-12900,38803,16,24,2.4,5.1,GeForce RTX 3090,26303,24.0,1560,1860,1,QHD,HIGH,96,117,179
2,High,Intel Core i9-12900,38803,16,24,2.4,5.1,GeForce RTX 3090,26303,24.0,1560,1860,1,QHD,MEDIUM,125,152,234
3,High,Intel Core i9-12900,38803,16,24,2.4,5.1,GeForce RTX 3080 Ti,26619,12.0,1365,1665,1,QHD,ULTRA,65,80,122
4,High,Intel Core i9-12900,38803,16,24,2.4,5.1,GeForce RTX 3080 Ti,26619,12.0,1365,1665,1,QHD,HIGH,88,108,162


In [12]:
frame_Y = pd.DataFrame(frame_data['GAME AVG FRAME'], columns = ['GAME AVG FRAME'])
frame_Y[:5]

,GAME AVG FRAME
0,87
1,117
2,152
3,80
4,108


In [13]:
scale_cols = ['CPU BenchMark', 'GPU BenchMark', 'CPU Core', 'CPU Thread', 'GPU Memory GB', 'GPU Clock Boost MHz', 'GAME NAME']
data_temp  = frame_data[scale_cols]
frame_X = pd.DataFrame(data_temp)

frame_X[:5]

,CPU BenchMark,GPU BenchMark,CPU Core,CPU Thread,GPU Memory GB,GPU Clock Boost MHz,GAME NAME
0,38803,26303,16,24,24.0,1860,1
1,38803,26303,16,24,24.0,1860,1
2,38803,26303,16,24,24.0,1860,1
3,38803,26619,16,24,12.0,1665,1
4,38803,26619,16,24,12.0,1665,1


In [14]:
frame_df = pd.concat([frame_Y, frame_X], axis=1)
frame_df[:5]

,GAME AVG FRAME,CPU BenchMark,GPU BenchMark,CPU Core,CPU Thread,GPU Memory GB,GPU Clock Boost MHz,GAME NAME
0,87,38803,26303,16,24,24.0,1860,1
1,117,38803,26303,16,24,24.0,1860,1
2,152,38803,26303,16,24,24.0,1860,1
3,80,38803,26619,16,24,12.0,1665,1
4,108,38803,26619,16,24,12.0,1665,1


In [16]:
def lin_regress(data, yvar, xvars):

    # output, input variables

    Y = data[yvar]
    X = data[xvars]

    # Create linear regression object

    linreg = linear_model.LinearRegression()

    # Fit the linear regression model

    model = linreg.fit(X, Y)
   
    # Get the intercept and coefficients

    intercept = model.intercept_
    coef = model.coef_
    result = [intercept, coef]

    return result

In [17]:
# GroupBy
grouped = frame_df.groupby('GAME NAME')

# Apply the UDF of linear regression model by Group
lin_reg_coef = grouped.apply(lin_regress, 'GAME AVG FRAME', ['CPU BenchMark', 'GPU BenchMark', 'CPU Core', 'CPU Thread', 'GPU Memory GB', 'GPU Clock Boost MHz'])


In [18]:
lin_reg_coef

GAME NAME
1    [-4.272886120375588, [0.0008256500214868153, 0...
2    [19.271672861686966, [-0.001000678182198969, 0...
3    [-35.76727689114614, [-0.0016285473058719982, ...
dtype: object

In [19]:
lin_reg_coef[1]
#CyberPunk frame  = -4.272886120375588 + 0.00082565 * CPU BenchMark + 0.00261913 * GPU BenchMark + 0.19640165 * CPU Core -0.23992268 * CPU Thread + 0.38713477 * GPU Memory GB + 0.00891777 * GPU Clock Boost MHz

[-4.272886120375588,
 array([ 0.00082565,  0.00261913,  0.19640165, -0.23992268,  0.38713477,
         0.00891777])]

In [31]:
a = lin_reg_coef[1]
type(a)


list

In [21]:
lin_reg_coef[2]
#League of Legends frame  = -35.76727689114614 -1.62854731e-03 * CPU BenchMark + 2.05895378e-02 * GPU BenchMark + 2.57089065e+01 * CPU Core -2.23463722e+01 * CPU Thread -2.96480871e+00 * GPU Memory GB + 1.64740117e-01 * GPU Clock Boost MHz

[19.271672861686966,
 array([-1.00067818e-03,  5.80172077e-03, -2.07565788e+00,  1.60445815e+00,
         1.07317453e-01,  2.86441349e-03])]

In [20]:
lin_reg_coef[3]
#PUBG frame  = 19.271672861686966 -1.00067818e-03 * CPU BenchMark + 5.80172077e-03 * GPU BenchMark -2.07565788e+00 * CPU Core + 1.60445815e+00 * CPU Thread + 1.07317453e-01 * GPU Memory GB + 2.86441349e-03 * GPU Clock Boost MHz

[-35.76727689114614,
 array([-1.62854731e-03,  2.05895378e-02,  2.57089065e+01, -2.23463722e+01,
        -2.96480871e+00,  1.64740117e-01])]